# Step 1. Making Preparations

In [50]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

import keras

In [29]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


# load datasets


In [51]:
# load dataset
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]



In [31]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)



# create baseline model

In [32]:
def baseline_model():
    model = Sequential()
    model.add(Dense(4,activation = 'relu',input_shape=(4,)))
    model.add(Dense(4,activation = 'relu'))
    model.add(Dense(3,activation = 'softmax'))
    model.compile(optimizer= 'adam',
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

In [33]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

# Evaluate The Model with k-Fold Cross Validation

In [34]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [35]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 96.67% (4.47%)


# Evaluate a Smaller Network

In [36]:
def smaller_model():
    model = Sequential()
    model.add(Dense(4, activation='relu', input_shape=(4,)))
    model.add(Dense(2, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model, write code below
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [37]:
estimator = KerasClassifier(build_fn=smaller_model, epochs=200, batch_size=5, verbose=0)

In [38]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [39]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 68.00% (32.22%)


# evalute a large network

In [40]:
def larger_model():
    model = Sequential()
    model.add(Dense(4, activation='relu', input_shape=(4,)))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(2, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model, write code below
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [41]:
estimator = KerasClassifier(build_fn=larger_model, epochs=200, batch_size=5, verbose=0)

In [42]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [43]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 58.67% (32.63%)


# functional model

In [52]:
def functional_model():
    inputs = keras.Input(shape=(4,))
    layer1 = Dense(4, activation='relu')(inputs)
    layer2 = Dense(4, activation='relu')(layer1)
    outputs = Dense(3, activation='softmax')(layer2)

    model = keras.Model(inputs, outputs)
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    return model

In [ ]:
estimator = KerasClassifier(build_fn=functional_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



# Rewriting the code using the Keras Functional API

In [46]:
from keras.layers import Input
from keras.layers import Dense
from keras.utils import plot_model
from keras.models import Model

Inputs = Input(shape=(4,))
x = Dense(4, activation='relu')(Inputs)
x = Dense(4, activation='relu')(x)
x = Dense(4, activation='relu')(x)
output = Dense(3, activation='softmax')(x)
model = Model(inputs=Inputs, outputs=output)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
dense_234 (Dense)            (None, 4)                 20        
_________________________________________________________________
dense_235 (Dense)            (None, 4)                 20        
_________________________________________________________________
dense_236 (Dense)            (None, 4)                 20        
_________________________________________________________________
dense_237 (Dense)            (None, 3)                 15        
Total params: 75
Trainable params: 75
Non-trainable params: 0
_________________________________________________________________
None


# Rewriting the code by doing Model Subclassing

In [47]:
import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = Dense(4, activation = 'relu')
        self.dense2 = Dense(4, activation = 'relu')
        self.dense3 = Dense(3, activation = 'softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)
    
model = MyModel()

# Rewriting the code without using scikit-learn

In [55]:
from sklearn.utils import shuffle

dataframe = shuffle(dataframe)

dataset = dataframe.values

X = dataset[:, 0:4].astype(float)
y = dataset[:, 4]

# label encoding
le = LabelEncoder()
encoded_y = le.fit(y).transform(y)

k=10
num_val_samples = len(X)//k
num_epochs = 200

all_acc = numpy.array([])
# all_err = np.array([])

In [56]:
for i in range(k):
    print('Processing K-Fold #', i+1)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_y[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = numpy.concatenate(
        [X[:i * num_val_samples],
         X[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = numpy.concatenate(
        [encoded_y[:i * num_val_samples],
         encoded_y[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = functional_model()
    history = model.fit(partial_train_data, partial_train_targets,
        epochs=num_epochs, batch_size=5, verbose=0, validation_data=(val_data, val_targets))
    
    results = model.evaluate(val_data, val_targets, verbose=0)
    
#     all_err = np.append(all_err, results[0])
    all_acc = numpy.append(all_acc, results[1])
    
print("+--------------------+")    
print("Custom K-Folds: Accuracy %.2f%%, Error (%.2f%%)" % (all_acc.mean()*100, all_acc.std()*100))

Processing K-Fold # 1
Processing K-Fold # 2
Processing K-Fold # 3
Processing K-Fold # 4
Processing K-Fold # 5
Processing K-Fold # 6
Processing K-Fold # 7
Processing K-Fold # 8
Processing K-Fold # 9
Processing K-Fold # 10
+--------------------+
Custom K-Folds: Accuracy 93.33%, Error (13.98%)
